In [1]:
# Imports
import numpy as np
import spikeinterface as si

In [2]:
# Properties of the in-memory dataset
num_channels=7
samplerate=30000
duration=20
num_timepoints=int(samplerate*duration)
num_units=4
num_events=1000

In [6]:
# Generate a pure-noise timeseries dataset and a linear geometry
timeseries=np.random.normal(0,10,(num_channels,num_timepoints))
geom=np.zeros((num_channels,2))
geom[:,0]=range(num_channels)

# Define the in-memory recording extractor
RX=si.NumpyRecordingExtractor(timeseries=timeseries,geom=geom,samplerate=samplerate)

# Generate some random events
times=np.sort(np.random.uniform(0,num_timepoints,num_events))
labels=np.random.randint(1,num_units+1,size=num_events)
    
# Define the in-memory sorting extractor
SX=si.NumpySortingExtractor()
for k in range(1,num_units+1):
    times_k=times[np.where(labels==k)[0]]
    SX.addUnit(unit_id=k,times=times_k)

In [11]:
# Demonstrate the API for extracting information
print('Unit ids = {}'.format(SX.getUnitIds()))
st=SX.getUnitSpikeTrain(unit_id=1)
print('Num. events for unit 1 = {}'.format(len(st)))
st1=SX.getUnitSpikeTrain(unit_id=1,start_frame=0,end_frame=30000)
print('Num. events for first second of unit 1 = {}'.format(len(st1)))

Unit ids = [1, 2, 3, 4]
Num. events for unit 1 = 268
Num. events for first second of unit 1 = 15


In [12]:
# Write the input/output in the MountainSort format
si.MdaRecordingExtractor.writeRecordedData(recording_extractor=IX,output_dirname='sample_mountainsort_dataset')
si.MdaSortingExtractor.writeSortedData(sorting_extractor=OX,firings_out='sample_mountainsort_dataset/firings_true.mda')

In [13]:
# Read this dataset with the Mda input extractor
RX2=si.MdaRecordingExtractor(dataset_directory='sample_mountainsort_dataset')
SX2=si.MdaSortingExtractor(firings_file='sample_mountainsort_dataset/firings_true.mda')

In [14]:
# We should get he same information as above
print('Unit ids = {}'.format(SX2.getUnitIds()))
st=SX2.getUnitSpikeTrain(unit_id=1)
print('Num. events for unit 1 = {}'.format(len(st)))
st1=SX2.getUnitSpikeTrain(unit_id=1,start_frame=0,end_frame=30000)
print('Num. events for first second of unit 1 = {}'.format(len(st1)))

Unit ids = [1 2 3 4]
Num. events for unit 1 = 239
Num. events for first second of unit 1 = 15


In [15]:
# For development purposes, reload imported modules when source changes
%load_ext autoreload
%autoreload 2

In [16]:
# Note that the following only works if you point the below path to the spike-collab directory

import sys, os
sys.path.append(os.getcwd()+'/../../spike-collab')
from widgets import TimeseriesWidget
from widgets import UnitWaveformsWidget

ImportError: cannot import name 'TimeseriesWidget' from 'widgets' (unknown location)

In [ ]:
TimeseriesWidget(input_extractor=IX).plot()

In [ ]:
UnitWaveformsWidget(input_extractor=IX2,output_extractor=OX2).plot()